In [ ]:
import numpy as np
import os.path
import pandas
import matplotlib
import scipy.signal
import scipy.constants

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec

# import mpld3
# from mpld3 import plugins

# data_dir = r"K:\ns\qt\Diamond\Projects\Cavities\Cavity characterisation paper\data\data_for_cav_char_paper\20160406\yoko_scans"
# data_dir = '/Users/wjwesterveld/Documents/Temp/' # copied above data to local temp folder, also to be sure we dont overwrite anyting
data_dir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170623/Scope' # copied above data to local temp folder, also to be sure we dont overwrite anyting




## Load xviewer csv (from binary scope data) and scope csv

In [ ]:
def read_scope_xviewer_csv(filename):
    headerRows = 10        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:                
                timeResolution = float( line[line.find(',')+1:line.rfind(',')].strip(' \t\n\r') )        
    data = pandas.read_csv(filename,header=None,skiprows=headerRows,usecols=[1]).values.flatten()
    return data, timeResolution


def read_scope_csv(filename):
    headerRows = 16        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:                
                timeResolution = float( line[line.rfind(',')+1:-1].strip(' \t\n\r') )    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows).values[:,1:-1]
    return data, timeResolution



file_name = "IFM_OFF007.csv"  
filename = os.path.join(data_dir,file_name)

data, timeResolution = read_scope_csv(filename)
time = timeResolution * np.arange(data.shape[0]) 

print '{} traces, time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
    data.shape[1], timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )


## Plot and Fit linewidth functions

In [ ]:
## Plot functions for linewidth measurements

def tmp_plot_function(time,data):
    fig = plt.figure(figsize=(16, 8))
    plt.subplot(121)
    plt.plot( time, data[:,1] )
    plt.plot( time, data[:,0])
    plt.xlabel('time (s)')
    plt.ylabel('signal (V)')    

    plt.subplot(122)    
    plt.plot( data[:,1], data[:,0])
    plt.xlabel('piezo signal (V)')
    plt.ylabel('cavity transmission (V)')    

    plt.tight_layout()    
    plt.show()

def tmp_read_and_plot(file_name,data_dir):    
    data, timeResolution = read_scope_csv(os.path.join(data_dir,file_name))
    time = timeResolution * np.arange(data.shape[0]) 
    print file_name
    tmp_plot_function(time,data)

In [ ]:
## LINEWIDTH IN GHz - FITTING AND ANALYSIS FUNCTIONS

import pylab
from scipy.optimize import leastsq # Levenberg-Marquadt Algorithm #

def lorentzian(x,p):    
    # Defined by wolfram, p[0] is center, p[1] is fwhm, p[3] is max intensity
    # See http://mathworld.wolfram.com/LorentzianFunction.html
    # But note that we have p[3] = 2 / (pi * fwhm) for integrated intensity of one.
    return p[2] * (0.5 * p[1])**2 / ( ( x - p[0] )**2 + ( 0.5 * p[1] )**2 )

def lorentzian3(x,p):  
    return p[2] * ( lorentzian(x,[p[0], p[1], 1]) + lorentzian(x,p[3:6]) + lorentzian(x,p[6:9]) )

def fit_lorenzian(time,data,fwhmGuess,timeResolution):
    # initial guess
    a1Guess = np.amax(data)
    t1Guess = np.argmax(data) * timeResolution + time[0]
    # fit lorenzian function to data
    def residuals(p,y,x):
        return y - lorentzian(x,p)
    guessParameters = [t1Guess,fwhmGuess,a1Guess]  # [peak center, hwhm, intensity]
    pbest = leastsq(residuals,guessParameters,args=(data,time),full_output=1)
    fitParameters = pbest[0]
    return fitParameters, guessParameters

def fit_lorenzian3(time,data,fwhmGuess,timeDeltaGuess,timeResolution):
    # initial guess
    a1Guess = np.amax(data) * 0.7
    a2Guess = 1
    a3Guess = 1
    t1Guess = np.argmax(data) * timeResolution + time[0]
    t2Guess = t1Guess + timeDeltaGuess
    t3Guess = t1Guess - timeDeltaGuess
    # Fit three lorenzians to data.
    def residuals(p,y,x):
        return y - lorentzian3(x,p)
    guessParameters = np.array([[t1Guess, fwhmGuess, a1Guess], [t2Guess, fwhmGuess, a2Guess], [t3Guess, fwhmGuess, a3Guess]]).flatten()
    pbest = leastsq(residuals,guessParameters,args=(data,time),full_output=1)
    fitParameters = pbest[0]
    return fitParameters, guessParameters
    


def fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess):
    ## Fit linewith
    # Load data
    data, timeResolution = read_scope_csv(filename1)
    time = timeResolution * np.arange(data.shape[0]) 
    i1 = int( time1 / timeResolution )
    i2 = int( time2 / timeResolution )
    data = data[i1:i2,0]
    time = time[i1:i2]
    # Fit
    transmissionFitParameters, transmissionGuessParameters = fit_lorenzian(time,data,fwhmGuess,timeResolution)
    # Plot
    fig = plt.figure(figsize=(16, 8))
    plt.subplot(121)    
    plt.plot( time, data )
    plt.plot( time, lorentzian(time,transmissionGuessParameters), linestyle=':' )
    plt.plot( time, lorentzian(time,transmissionFitParameters) )    
    print 'peak position {} s, fwhm {} s, intensity {} V'.format( \
        transmissionFitParameters[0], transmissionFitParameters[1], transmissionFitParameters[2] )

    ## Fit scanning speed with modulated signal 6 GHz
    # Load data
    data, timeResolution = read_scope_csv(filename2)
    time = timeResolution * np.arange(data.shape[0]) 
    i1 = int( time3 / timeResolution )
    i2 = int( time4 / timeResolution )
    data = data[i1:i2,0]
    time = time[i1:i2]
    # Fit
    fitParameters, guessParameters = fit_lorenzian3(time,data,fwhmGuess,timeDeltaGuess,timeResolution)
    # Plot        
    plt.subplot(122)
    plt.plot( time, data )
    plt.plot( time, lorentzian3(time,guessParameters), linestyle=':' )
    plt.plot( time, lorentzian3(time,fitParameters) )
    plt.show()
        
    print 'peak positions (s)', fitParameters[[0,3,6]]
    print 'peak fwhm (s)', fitParameters[[1,4,7]]
    print 'peak intensity (V)', fitParameters[[2]], fitParameters[[2]] * fitParameters[[5]], \
         fitParameters[[2]] * fitParameters[[8]]
        
    dtSideModulationPeaks = np.amax( fitParameters[[0,3,6]] ) - np.amin( fitParameters[[0,3,6]] )
    dfOverDt = 12.0e9 / dtSideModulationPeaks    
    print 'df/dt = 12/{} GHz/s = {} THz/s'.format( \
        dtSideModulationPeaks, dfOverDt *1e-12 )
    cavityFwhmInS = transmissionFitParameters[1] * dfOverDt
    print 'cavity fwhm = {} s = {} GHz'.format( transmissionFitParameters[1], cavityFwhmInS*1e-9 )
    
    




## Plot and fit linewidth measurements Mon 28 Aug 2017

In [ ]:
data_dir = r'D:\measuring\data\20170828\125800_cav_linewidth'

# Update data dir
scope_dir = r'O:\WOUTER\20170828'
import os, shutil
files = [file for file in os.listdir(scope_dir) if os.path.isfile(os.path.join(scope_dir, file))]
for file in files:
    if not os.path.exists(os.path.join(data_dir, file)):
        shutil.copy(os.path.join(scope_dir, file), data_dir)
 
# Plot files
fig = plt.figure(figsize=(16, 8))
for i in [20, 21]:
    file_name = '0828_{:03d}.csv'.format(i)
    data, timeResolution = read_scope_csv( os.path.join( data_dir, file_name ) )    
    time = timeResolution * np.arange( data.shape[0] )    
    plt.plot( time, data[:,0], label=file_name )
    print i
plt.grid()
plt.legend()
plt.show()


# Fit linewidth
time1 = 0.003
time2 = time1 + 0.006
fwhmGuess = 0.0005
for i in range(26,57,2):
    file_name = '0828_{:03d}.csv'.format(i)
    data, timeResolution = read_scope_csv( os.path.join( data_dir, file_name ) )    
    time = timeResolution * np.arange( data.shape[0] )   
    i1 = int( time1 / timeResolution )
    i2 = int( time2 / timeResolution )
    data = data[i1:i2,0]
    time = time[i1:i2]
    # Fit
    transmissionFitParameters, transmissionGuessParameters = fit_lorenzian(time,data,fwhmGuess,timeResolution)
    # Plot
    fig = plt.figure(figsize=(16, 8))
    plt.subplot(121)    
    plt.plot( time, data )
    plt.plot( time, lorentzian(time,transmissionGuessParameters), linestyle=':' )
    plt.plot( time, lorentzian(time,transmissionFitParameters) )    
    print '{:s}, fwhm {:.3f} ms, intensity {:.3f} V'.format( file_name, transmissionFitParameters[1]*1e3, transmissionFitParameters[2] )



     

    
    

In [ ]:
   
    
# Fit linewidth
time1 = 0.003
time2 = time1 + 0.006
fwhmGuess = 0.0005
timeResolution = 1.6e-06
NCentered = 1000
timeCentered = np.arange(-NCentered,NCentered) * timeResolution   
for i in [36, 38, 58, 60, 62, 64]:    
    file_name = '0828_{:03d}.csv'.format(i)
    data, timeResolution = read_scope_csv( os.path.join( data_dir, file_name ) )    
    time = timeResolution * np.arange( data.shape[0] )   
    i1 = int( time1 / timeResolution )
    i2 = int( time2 / timeResolution )
    data = data[i1:i2,0]
    time = time[i1:i2]                
    iMax = np.argmax( data )    
    plt.plot( timeCentered, data[iMax-NCentered:iMax+NCentered], label=file_name )  
    file_name = '0828_{:03d}.csv'.format(i+1)
    data, timeResolution = read_scope_csv( os.path.join( data_dir, file_name ) )    
    time = timeResolution * np.arange( data.shape[0] )   
    i1 = int( time1 / timeResolution )
    i2 = int( time2 / timeResolution )
    data = data[i1:i2,0]
    time = time[i1:i2]    
    iMax = np.argmax( data )    
    plt.plot( timeCentered, data[iMax-NCentered:iMax+NCentered], label=file_name )   
    plt.grid()
    plt.legend()
    plt.show();

## Plot and fit linewidth measurements Fri 23 June 2017

In [ ]:
## Plot linewidth measurements 23 June 2017

data_dir = 'TODO'

tmp_read_and_plot('IFM_OFF006.csv',data_dir)
tmp_read_and_plot('IFM_OFF007.csv',data_dir)


tmp_read_and_plot('IFM_OFF009.csv',data_dir)
tmp_read_and_plot('IFM_OFF013.csv',data_dir)


tmp_read_and_plot('IFM_OFF012.csv',data_dir)
tmp_read_and_plot('IFM_OFF010.csv',data_dir)

tmp_read_and_plot('IFM_OFF011.csv',data_dir)




In [ ]:
## Plot single file with mpld3 enabled

data, timeResolution = read_scope_csv(os.path.join(data_dir,'IFM_OFF010.csv'))
time = timeResolution * np.arange(data.shape[0]) 

import mpld3
from mpld3 import plugins
mpld3.enable_notebook()
fig = plt.figure(figsize=(13, 8))
plt.plot( time, data[:,0] )
plt.grid()
plt.xlim([0, 0.005])
plugins.connect(fig, plugins.MousePosition(fontsize=14))
plt.show()
mpld3.disable_notebook()


In [ ]:
# results
# 
# Method
#  assume linear sweep around resonance
#  measure fwhm and GHz modulation peak distances by hand.
#  Use 6 GHz modulation to estimate df/dt. I expect this to be better than using the linescan measurements as the sweep range there is much larger and not using AWG.
# Results
#  File 6, 7 (40 Hz piezo sweep)
#     File 6, df = 12 GHz, dt = 0.0028 s. 
#     File 7, max = 0.85 V, min = 0.1 V, hm = 0.475, fw ~ 0.002 s 
#     Linewidth 8.6 GHz, measurement time 0.015 s.
#  File 9, 13 (80 Hz piezo sweep)
#     File 13, df = 12 GHz, dt = 0.0037 - 0.0022 = 0.0015 s. 
#     File 9, max = 0.8 V, min = 0.05 V, hm = 0.425, fw ~ 0.001 s 
#     Linewidth 8 GHz, measurement time 0.005 s (200 Hz)
#  File 10, 12 (200 Hz piezo sweep)
#     File 13, df = 12 GHz, dt = 7e-4 s. 
#     File 9, hm = 0.39, fw ~ 5.1e-4 s 
#     Linewidth 8.7 GHz, measurement time 0.0025 s (400 Hz)
#
# Ok, so we can use 40 80, or 200 Hz for linewidht measurements. Linewidht is not affected by piezo sweeping speed or by lag.
# I do not think we can use the 400 Hz for linewidht measurement because the signal does not go to zero between sweeps.











In [ ]:
## LINEWIDTH IN GHz - DATA 23 June 2017

data_dir = '/Users/wjwesterveld/Documents/Temp/20170623/Scope' # copied above data to local temp folder, also to be sure we dont overwrite anyting


# Fri June 23th, 40 Hz sweeping speed
filename1 = os.path.join(data_dir,'IFM_OFF007.csv')
time1 = 0.005
time2 = 0.015
filename2 = os.path.join(data_dir,'IFM_OFF006.csv')
time3 = 0.005
time4 = 0.013
fwhmGuess = 0.001
timeDeltaGuess = 0.001
print '40 Hz sweeping speed, file1 ', os.path.basename(filename1), ', file2 ', os.path.basename(filename2)
fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess)

# fit   [ 0.00901127  0.0105146   0.00713837  ] for timeDelta = 0.003  
# fit   [ 0.00901126  0.01051459  0.00713838]   for timeDelta = 0.002
# fit   [ 0.00901127  0.0105146   0.00713837]   for timeDelta = 0.001



# Fri June 23th, 80 Hz sweeping speed
filename1 = os.path.join(data_dir,'IFM_OFF009.csv')
time1 = 0.000
time2 = 0.006
filename2 = os.path.join(data_dir,'IFM_OFF013.csv')
time3 = 0.000
time4 = 0.005
fwhmGuess = 0.002
timeDeltaGuess = 0.001
print ''
print ''
print '80 Hz sweeping speed, file1 ', os.path.basename(filename1), ', file2 ', os.path.basename(filename2)
fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess)

# Fri June 23th, 200 Hz sweeping speed
filename1 = os.path.join(data_dir,'IFM_OFF010.csv')
time1 = 0.001
time2 = 0.003
filename2 = os.path.join(data_dir,'IFM_OFF012.csv')
time3 = 0.001
time4 = 0.003
fwhmGuess = 0.001
timeDeltaGuess = 0.0001
print ''
print ''
print '200 Hz sweeping speed, file1 ', os.path.basename(filename1), ', file2 ', os.path.basename(filename2) 
# mpld3.enable_notebook().
fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess)
# mpld3.disable_notebook()

## PLOT and FIT Linewidth measurements 28 June 2017

In [ ]:
# Plot linewidth measurements 28 June 2017

data_dir = '/Users/wjwesterveld/Documents/Temp/20170628/scope_DLM4038'

tmp_read_and_plot('IFM_OFF016.csv',data_dir)
tmp_read_and_plot('IFM_OFF018.csv',data_dir)

tmp_read_and_plot('IFM_OFF020.csv',data_dir)
tmp_read_and_plot('IFM_OFF019.csv',data_dir)

tmp_read_and_plot('IFM_OFF021.csv',data_dir)
tmp_read_and_plot('IFM_OFF023.csv',data_dir)


In [ ]:
## LINEWIDTH IN GHz - DATA 28 June 2017

data_dir = '/Users/wjwesterveld/Documents/Temp/20170628/scope_DLM4038'

# 28 June 2017, 40 GHz
filename1 = os.path.join(data_dir,'IFM_OFF016.csv')
time1 = 0.00
time2 = 0.01
filename2 = os.path.join(data_dir,'IFM_OFF018.csv')
time3 = 0.00
time4 = 0.01
fwhmGuess = 0.001
timeDeltaGuess = 0.001
print ''
print ''
print '40 Hz sweeping speed, file1 ', os.path.basename(filename1), ', file2 ', os.path.basename(filename2) 
fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess)


# 28 June 2017, 40 GHz
filename1 = os.path.join(data_dir,'IFM_OFF020.csv')
time1 = 0.009
time2 = 0.021
filename2 = os.path.join(data_dir,'IFM_OFF019.csv')
time3 = 0.009
time4 = 0.021
fwhmGuess = 0.002
timeDeltaGuess = 0.002
print ''
print ''
print '40 Hz sweeping speed, 3 Vpp, file1 ', os.path.basename(filename1), ', file2 ', os.path.basename(filename2) 
fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess)


# 28 June 2017, 200 GHz
filename1 = os.path.join(data_dir,'IFM_OFF021.csv')
time1 = 0.000
time2 = 0.002
filename2 = os.path.join(data_dir,'IFM_OFF023.csv')
time3 = 0.000
time4 = 0.002
fwhmGuess = 0.0002
timeDeltaGuess = 0.0002
print ''
print ''
print '200 Hz sweeping speed, 6.5 Vpp, file1 ', os.path.basename(filename1), ', file2 ', os.path.basename(filename2) 
fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess)



In [ ]:
a = np.arange(10)
print a
print a[0]
print a[[0,2,9]]

## Plot cavity transmittance measurements

In [ ]:
def tmp_plot_vibrations(data,time,time1,time2):
    timeResolution = time[1] - time[0]
    i1 = int( time1 / timeResolution )
    i2 = int( time2 / timeResolution )
    
    freq_pow, power = scipy.signal.welch(data[i1:i2], axis = 0, fs = 1/timeResolution,nperseg = i2-i1, detrend = 'linear' )
    j1 = np.argmax(freq_pow>0)
    j2 = np.argmax(freq_pow>200)        
    
    fig = plt.figure(figsize=(16, 8))

    plt.subplot(121)
    plt.plot( time[i1:i2], data[i1:i2] )   
    plt.xlabel('time (s)')
    plt.ylabel('signal (V)')  
        
    plt.subplot(122)    
    plt.plot(freq_pow[j1:j2], power[j1:j2])
    # plt.ylim([0,0.01])
    plt.xlabel('freq (Hz)')
    plt.ylabel('Noise power (V**2/Hz)')
    
    plt.show()  




In [ ]:
# Find point of maximal sensitivity
# assume off-resonance transmittance corresponds to 0 V
# assume maximal transmission corresponds to maximum value in time trace

import scipy.constants

def find_drift_time_of_maximal_sensitivity(timeCoarse,dataCoarse):
    """
    Return time when data is above 0.75% of its maximal value. 
    We assume a measurement where the cavity length drifts slowly over its resonance. Thus
    maximum value of data is transmission of cavity at resonance. Sensitivity of Lorenzian
    shape is maximum the flank, when it is at 0.75 of its maxiumum.     
    """
    iFlank = np.argmax(dataCoarse > 0.75 * np.amax( data ) )
    return timeCoarse[iFlank]
    
def find_df_over_dV(fwhm,dataMax):
    """
    Return tangent of Lorenzian lineshape at the position where the flank is steepest. This
    is d frequency / d voltage.
    
    fwhm, full-width at half-max of cavity resonance, Hz
    dataMax, maximum amplitude of data, i.e., amplitude of transmission at resonance, V
    """
    return (4 * fwhm ) / ( dataMax * 3 * np.sqrt(3) )
    
def find_dl_over_df(centerFrequency,cavityLength):
    """
    Return d cavity_length / d frequency 
    
    centerFrequeny, laser frequency used in measurement, Hz
    cavityLength, m
    """
    return -cavityLength / centerFrequency    
    

def load_drift_vibration_trace(dataDir, fileName, time1 = 0):    
    data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
    time = timeResolution * np.arange(data.shape[0]) 
    print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )
    # crop data
    i1 = int( time1 / timeResolution )    
    time = time[i1:]
    data = data[i1:]    
    # Apply polynomial smoothing to get timeCoarse, dataCoarse
    p = np.polynomial.polynomial.polyfit(time,data,9)
    timeCoarse = np.arange(time[0],time[-1],0.1)
    dataCoarse = np.polynomial.polynomial.polyval(timeCoarse,p)
    return time, data, timeCoarse, dataCoarse
    
def analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow ):
    
    # Find point of maximum sensitivity: 0.75 * dataMax
    dataMax = np.amax( data )
    tFlank = find_drift_time_of_maximal_sensitivity(timeCoarse,dataCoarse)
    dfOverdV = find_df_over_dV(fwhm,dataMax)
    dlOverdV = find_dl_over_df(centerFrequency,cavityLength) * dfOverdV
    
    print 'dataMax', dataMax
    print 'dV/df = ', 1.0 / dfOverdV
    print 'dl/df = ', find_dl_over_df(centerFrequency,cavityLength)    

    # Plot
    fig = plt.figure(figsize=(16, 8))
    plt.title( 'drifted to maximal sensitivity at {} s'.format(tFlank) )
    plt.plot( time, data, label='data' )   
    plt.plot( timeCoarse, dataCoarse, label='polynoimal smoothening' )    
    plt.plot( [tFlank, tFlank], plt.ylim(), label='0.75 * max' )
    plt.xlabel('time (s)')
    plt.ylabel('signal (V)')    
    plt.grid()
    plt.show()    

    # For a Lorenzian line shape, maximum dV/df = A*3*sqrt(3) / (4 * fwhm ), with Lorenz maximum amplitude A [V] and fwhm [m].
    # Around frequeny f, we find that the linear relation between resonance frequency f_res and cavity length l, df_res / dl = f / l.

    # Select time window, apply linear detrend, compute RMS
    timeResolution = time[1] - time[0]
    i1 = int( (tFlank - 0.5 * timeWindow - time[0]) / timeResolution )
    i2 = int( (tFlank + 0.5 * timeWindow - time[0]) / timeResolution )
    t = time[i1:i2]
    d = data[i1:i2]
    d = d - np.polynomial.polynomial.polyval(t,np.polynomial.polynomial.polyfit(t,d,1))
    rmsV = np.sqrt(np.mean(d**2))

    # Plot        
    fig = plt.figure(figsize=(16, 8))
    plt.title('time window around flank max, linear detrend, rms {:.3f} V, {:.3f} GHz, {:.3f} nm'.format( rmsV, dfOverdV * rmsV * 1e-9, -dlOverdV * rmsV * 1e9 ) )
    plt.plot(t, dlOverdV*d*1e9)
    plt.xlabel('time (s)')
    plt.ylabel('detuning (nm)')
    plt.show()

    freq_pow, power = scipy.signal.welch(1e9*dlOverdV*d, axis = 0, fs = 1/timeResolution,nperseg = i2-i1, detrend = 'linear' )
    j1 = np.argmax(freq_pow>0)
    j2 = np.argmax(freq_pow>250)    
    fig = plt.figure(figsize=(16, 8))
    plt.plot(freq_pow[j1:j2], 10.0*np.log10( power[j1:j2] ) )
    plt.grid()
    plt.xlabel('freq (Hz)')
    plt.ylabel('Noise power (nm**2/Hz in dB)')
    plt.show()  

## TODO
## Figure out which measurements I have done and which results I would like to get from there
# Proof that these are cavity vibrations V
#  - Red at differnt positinos V
#  - Green V
# Quantify amplitude of vibrations
# Analyze differnt connections of piezo's




In [ ]:
# Example of functions in previous cell


print 'red: vibrations'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170628/scope_DLM4038'
fileName = "IFM_OFF017.wdf.csv"  

timeWindow = 2       # s
fwhm = 8*1e9         # Hz
cavityLength = 15e-6 # m
centerFrequency = scipy.constants.c / 637.0e-9 # Hz

# time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )



## Analyze data 28 June 2017, show that 100 Hz signal is cavity vibration (or laser phase noise)

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170628/scope_DLM4038'

if False:
    titleStr = 'red: vibrations'    
    fileName = "IFM_OFF017.wdf.csv"  
if True:    
    titleStr = 'green: optical setup path vibrations'    
    fileName = "IFM_OFF007.wdf.csv"  
if False:
    titleStr = 'black (no light): photo-diode electronics'
    fileName = "IFM_OFF009.wdf.csv"  


print titleStr
time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)   

In [ ]:
print titleStr

timeWindow = 2       # s

fig = plt.figure(figsize=(16, 20))
for i in range(3):
    if i == 0:
        timeWindowTime = 75                    
    elif i == 1:
        timeWindowTime = 90        
    elif i == 2:
        timeWindowTime = 10        
    labelStr = '{:.0f} s around {:.0f} s'.format(timeWindow, timeWindowTime)    
    timeResolution = time[1] - time[0]
    i1 = int( (timeWindowTime - 0.5 * timeWindow ) / timeResolution )
    i2 = int( (timeWindowTime + 0.5 * timeWindow ) / timeResolution )
    t = time[i1:i2]
    d = data[i1:i2]
    d = d - np.polynomial.polynomial.polyval(t,np.polynomial.polynomial.polyfit(t,d,1))
    rmsV = np.sqrt(np.mean(d**2))
    freq_pow, power = scipy.signal.welch(d*1e3, axis = 0, fs = 1/timeResolution,nperseg = i2-i1 )
    j1 = np.argmax(freq_pow>0)
    j2 = np.argmax(freq_pow>250)   
    # j2 = np.argmax(freq_pow>1500)   
    
    # Plot            
    plt.subplot(4,2,i*2+1)
    plt.plot(t,d)
    plt.title('time window {}, linear detrend, rms {:.3f} mV'.format( labelStr, rmsV*1e3) )                    
    plt.xlabel('time (s)')
    plt.ylabel('signal (mV)')
    plt.grid()     
    
    plt.subplot(4,2,i*2+2)
    plt.plot(freq_pow[j1:j2], power[j1:j2], label=labelStr )    
    plt.xlabel('freq (Hz)')
    plt.ylabel('Noise power (mV**2/Hz)')
    plt.grid()
    
    plt.subplot(4,2,7)
    plt.plot( [timeWindowTime, timeWindowTime], [0, 1100], label=labelStr )
    
    plt.subplot(4,2,8)
    plt.plot(freq_pow[j1:j2], power[j1:j2], label=labelStr )            
    plt.xlabel('freq (Hz)')
    plt.ylabel('Noise power (mV**2/Hz)')
    plt.legend()
    plt.grid()

    
plt.subplot(4,2,7)
plt.plot( time, data*1e3, label='trace' )
plt.xlabel('time (s)')
plt.ylabel('signal (mV)') 
plt.ylim( 0, 1100 )
plt.legend()
plt.grid()
    
plt.show() 






In [ ]:
print titleStr

timeWindow = 2       # s
timeWindowTime = 75  # s

fig = plt.figure(figsize=(10, 8))
   
labelStr = '{:.0f} s around {:.0f} s'.format(timeWindow, timeWindowTime)    
timeResolution = time[1] - time[0]
i1 = int( (timeWindowTime - 0.5 * timeWindow ) / timeResolution )
i2 = int( (timeWindowTime + 0.5 * timeWindow ) / timeResolution )
t = time[i1:i2]
d = data[i1:i2]
d = d - np.polynomial.polynomial.polyval(t,np.polynomial.polynomial.polyfit(t,d,1))
rmsV = np.sqrt(np.mean(d**2))
freq_pow, power = scipy.signal.welch(d*1e3, axis = 0, fs = 1/timeResolution,nperseg = i2-i1 )
j1 = np.argmax(freq_pow>0)
j2 = np.argmax(freq_pow>250)   
j3 = np.argmax(freq_pow>1500)   
    
# Plot      
plt.subplot(2,2,1)
plt.plot( time, data*1e3, label='trace' )
plt.plot( [timeWindowTime, timeWindowTime], [0, 1100], label=labelStr )
plt.xlabel('time (s)')
plt.ylabel('signal (mV)') 
plt.ylim( 0, 1100 )
plt.legend()
plt.grid()

plt.subplot(2,2,2)
plt.plot(t,d)
plt.title('time window {}, linear detrend, rms {:.3f} mV'.format( labelStr, rmsV*1e3) )                    
plt.xlabel('time (s)')
plt.ylabel('signal (mV)')
plt.grid()     

plt.subplot(2,2,3)
plt.plot(freq_pow[j1:j3], power[j1:j3], label=labelStr )    
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (mV**2/Hz)')
plt.grid()

plt.subplot(2,2,4)
plt.plot(freq_pow[j1:j2], power[j1:j2], label=labelStr )    
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (mV**2/Hz)')
plt.grid()    
    
plt.show() 

## Analyze data 29 and 30 June 2017, try to reduce 100 Hz signal

In [ ]:
## Analyze data 

# 29 6 2017.
# ~4 V fine-piezo

#018.wdf -> laser power 1.5 mW, same as yesterday.
#019.wdf -> laser power 1.5 mW, without laser piezo fine control wire connected.
#021.wdf -> laser power 1.5 mW, without laser piezo fine control wire connected. JPE without coarse piezo connected.
#023.wdf -> laser power 1.5 mW, without laser piezo fine control wire connected. JPE without coarse piezo connected. Unplug loose wires from voltage dividers.


# 30 6 2017
# ~0V fine-piezo

#030.WDF. Drift across resonance. Last step was decreasing voltage piezo C. Piezo A at 0V, Piezo B at 0V, Piezo C with 1 Hz low-pass filter at 0.968 V.
#032.WDF. Drift across resonance. Last step was decreasing voltage piezo C. Piezo A control wire disconnected, Piezo B control wire disconnected, Piezo C with 1 Hz low-pass filter at 0.968 V.
#035.WDF. Drift across resonance. Last step was increase in voltage piezo C. Piezo A piezo (controller output) wire disconnected, Piezo B control wire piezo (controller output) disconnected, Piezo C directly at 0.889 V. Al unnsecary wires disconnected from ADWIN.

# Scanner sensitivity @ 293 K: 25 nm/V --> 4V decreas is 0.1 micrometer --> within cavity length uncertainty.

timeWindow = 2       # s
fwhm = 8*1e9         # Hz
cavityLength = 15e-6 # m
centerFrequency = scipy.constants.c / 637.0e-9 # Hz

In [ ]:
print '#018.wdf -> laser power 1.5 mW, same as yesterday.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170629/scope'
fileName = "IFM_OFF018.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName, 10)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )


In [ ]:
print '#019.wdf -> laser power 1.5 mW, without laser piezo fine control wire connected.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170629/scope'
fileName = "IFM_OFF019.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )

In [ ]:
print '#021.wdf -> laser power 1.5 mW, without laser piezo fine control wire connected. JPE without coarse piezo connected.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170629/scope'
fileName = "IFM_OFF021.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )


In [ ]:
print '#023.wdf -> laser power 1.5 mW, without laser piezo fine control wire connected. JPE without coarse piezo connected. Unplug loose wires from voltage dividers.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170629/scope'
fileName = "IFM_OFF023.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )

In [ ]:
print '#028.WDF. We have extremely low drift now. Steps are manually increasing and decreasing voltage on piezo C.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170630/scope'
fileName = "IFM_OFF027.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName, 5)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )


In [ ]:
print '#030.WDF. Drift across resonance. Last step was decreasing voltage piezo C. Piezo A at 0V, Piezo B at 0V, Piezo C with 1 Hz low-pass filter at 0.968 V.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170630/scope'
fileName = "IFM_OFF030.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName, 20)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )

In [ ]:
print '#031.WDF. Drift across resonance. Last step was decreasing voltage piezo C. Piezo A control wire disconnected, Piezo B at 0V, Piezo C with 1 Hz low-pass filter at 0.968 V.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170630/scope'
fileName = "IFM_OFF031.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName )    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )

In [ ]:
print '#032.WDF. Drift across resonance. Last step was decreasing voltage piezo C. Piezo A control wire disconnected, Piezo B control wire disconnected, Piezo C with 1 Hz low-pass filter at 0.968 V.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170630/scope'
fileName = "IFM_OFF032.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )


In [ ]:
print '#033.WDF. Drift across resonance. Last step was decreasing voltage piezo C. Piezo A piezo (controller output) wire disconnected, Piezo B control wire piezo (controller output) disconnected, Piezo C with 1 Hz low-pass filter at 0.968 V.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170630/scope'
fileName = "IFM_OFF033.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )

In [ ]:
print '#034.WDF. Drift across resonance. Last step was increase in voltage piezo C. Piezo A piezo (controller output) wire disconnected, Piezo B control wire piezo (controller output) disconnected, Piezo C directly at 0.889 V. '
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170630/scope'
fileName = "IFM_OFF034.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )

In [ ]:
print '#035.WDF. Drift across resonance. Last step was increase in voltage piezo C. Piezo A piezo (controller output) wire disconnected, Piezo B control wire piezo (controller output) disconnected, Piezo C directly at 0.889 V. Al unnsecary wires disconnected from ADWIN.'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170630/scope'
fileName = "IFM_OFF035.wdf.csv"  

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )

## Vibration measurement 29 Aug 2017 with new mirrors


In [ ]:
dataDir = r'D:\measuring\data\20170829\scope'
fileName = "SCOPE_003.wdf.csv"  

timeWindow = 2        # s
fwhm = 6.81e9         # Hz
cavityLength = 9.7e-6 # m
centerFrequency = scipy.constants.c / 635.6e-9 # Hz

time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )

## OLD Stuff

In [ ]:
# Load data
data_dir = '/Users/wjwesterveld/Documents/Temp/20170629/scope'
fileNumbers = [18, 19, 20, 21, 23]

for i in range( len( fileNumbers ) ):
    filename = os.path.join(data_dir,'IFM_OFF{:03d}.wdf.csv'.format( fileNumbers[i] )  )
    data1, timeResolution = read_scope_xviewer_csv(filename)
    time = timeResolution * np.arange(data.shape[0])  
    if i == 0:
        data = np.zeros( ( len(data), len(fileNumbers) ) )    
    data[:,i] = data1
    

print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
    timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )

In [ ]:
# Plot data

for i in range( len(fileNumbers) ):
    fig = plt.figure(figsize=(16, 8))
    plt.plot( time, data[:,i] )
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'frequency (Hz)' )
    plt.grid()
    plt.show()
    


